# ガードレールの使用方法

このノートブックでは、LLMアプリケーションにガードレールを実装する方法の例を紹介します。ガードレールは、アプリケーションを適切な方向に導くことを目的とした**検出制御**の総称です。LLMに固有のランダム性を考慮すると、より高い制御性は一般的な要件であり、効果的なガードレールの作成は、LLMをプロトタイプから本番環境に移行する際の最も一般的なパフォーマンス最適化領域の一つとなっています。

ガードレールは非常に[多様](https://github.com/NVIDIA/NeMo-Guardrails/blob/main/examples/README.md)で、LLMで問題が発生する可能性があると想像できるほぼすべてのコンテキストに展開できます。このノートブックは、あなた独自のユースケースに合わせて拡張できる簡単な例を提供し、ガードレールを実装するかどうかを決定する際に考慮すべきトレードオフと、その実装方法を概説することを目的としています。

このノートブックでは以下に焦点を当てます：
1. **入力ガードレール** - 不適切なコンテンツがLLMに到達する前にフラグを立てる
2. **出力ガードレール** - LLMが生成したものが顧客に届く前に検証する

**注意：** このノートブックでは、ガードレールをLLM周辺の検出制御の総称として扱います。事前構築されたガードレールフレームワークの配布を提供する公式ライブラリについては、以下をご確認ください：
- [NeMo Guardrails](https://github.com/NVIDIA/NeMo-Guardrails/tree/main)
- [Guardrails AI](https://github.com/ShreyaR/guardrails)

In [20]:
import openai

GPT_MODEL = 'gpt-4o-mini'

## 1. 入力ガードレール

入力ガードレールは、不適切なコンテンツがLLMに到達することを最初から防ぐことを目的としています。一般的な使用例は以下の通りです：
- **トピック別ガードレール：** ユーザーがトピック外の質問をした場合を特定し、LLMがサポートできるトピックについてアドバイスを提供します。
- **ジェイルブレイキング：** ユーザーがLLMを乗っ取り、そのプロンプトを上書きしようとしている場合を検出します。
- **プロンプトインジェクション：** ユーザーがLLMが実行する下流の関数で実行される悪意のあるコードを隠そうとするプロンプトインジェクションのインスタンスを検出します。

これらすべてにおいて、予防的制御として機能し、LLMの前または並行して実行され、これらの基準のいずれかが満たされた場合にアプリケーションが異なる動作をするようトリガーします。

### ガードレールの設計

ガードレールを設計する際は、**精度**、**レイテンシ**、**コスト**のトレードオフを考慮することが重要です。ここでは、収益とユーザーエクスペリエンスへの影響を最小限に抑えながら、最大の精度を達成することを目指します。

まず、トピック外の質問を検出し、トリガーされた場合にLLMが回答することを防ぐことを目的とした、シンプルな**トピック別ガードレール**から始めます。このガードレールはシンプルなプロンプトで構成され、`gpt-4o-mini`を使用してレイテンシ/コストを最大化しながら十分な精度を保持しますが、さらに最適化したい場合は以下を検討できます：
- **精度：** `gpt-4o-mini`のファインチューニングやfew-shotの例を検討して精度を向上させることができます。コンテンツが許可されているかどうかを判断するのに役立つ情報のコーパスがある場合、RAGも効果的です。
- **レイテンシ/コスト：** `babbage-002`やLlamaなどのオープンソースの提供物など、より小さなモデルのファインチューニングを試すことができます。これらは十分な訓練例が与えられた場合、非常に良いパフォーマンスを発揮できます。オープンソースの提供物を使用する場合、推論に使用するマシンを調整して、コストまたはレイテンシの削減のいずれかを最大化することもできます。

このシンプルなガードレールは、LLMが事前定義されたトピックセットにのみ回答し、範囲外のクエリには定型メッセージで応答することを確実にすることを目的としています。

### 非同期の活用

レイテンシを最小化する一般的な設計は、メインのLLM呼び出しと並行してガードレールを非同期で送信することです。ガードレールがトリガーされた場合はその応答を返し、そうでなければLLMの応答を返します。

このアプローチを使用して、LLMの`get_chat_response`と`topical_guardrail`ガードレールを並行して実行し、ガードレールが`allowed`を返す場合のみLLMの応答を返す`execute_chat_with_guardrails`関数を作成します。

### 制限事項

設計を開発する際は、常にガードレールの制限事項を考慮する必要があります。認識すべき主要な制限事項は以下の通りです：
- LLMをガードレールとして使用する場合、ベースのLLM呼び出し自体と同じ脆弱性を持つことに注意してください。例えば、**プロンプトインジェクション**の試みは、ガードレールと実際のLLM呼び出しの両方を回避することに成功する可能性があります。
- 会話が長くなるにつれて、指示が追加のテキストによって希釈されるため、LLMは**ジェイルブレイキング**により影響を受けやすくなります。
- 上記の問題を補償するためにガードレールを過度に制限的にすると、ユーザーエクスペリエンスを損なう可能性があります。これは**過度の拒否**として現れ、プロンプトインジェクションやジェイルブレイキングの試みとの類似性があるため、ガードレールが無害なユーザーリクエストを拒否します。

### 軽減策

ガードレールをルールベースまたはより従来の機械学習モデルと組み合わせて検出に使用できれば、これらのリスクの一部を軽減できます。また、長い会話によってモデルが混乱するリスクを軽減するために、最新のメッセージのみを考慮するガードレールを使用している顧客も見てきました。

また、プロンプトインジェクションやジェイルブレイキングのインスタンスを検出できるよう、会話の積極的な監視を伴う段階的なロールアウトを行うことをお勧めします。これにより、これらの新しいタイプの行動をカバーするためにより多くのガードレールを追加するか、既存のガードレールの訓練例として含めることができます。

In [21]:
system_prompt = "You are a helpful assistant."

bad_request = "I want to talk about horses"
good_request = "What are the best breeds of dog for people that like cats?"

In [22]:
import asyncio


async def get_chat_response(user_request):
    print("Getting LLM response")
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_request},
    ]
    response = openai.chat.completions.create(
        model=GPT_MODEL, messages=messages, temperature=0.5
    )
    print("Got LLM response")

    return response.choices[0].message.content


async def topical_guardrail(user_request):
    print("Checking topical guardrail")
    messages = [
        {
            "role": "system",
            "content": "Your role is to assess whether the user question is allowed or not. The allowed topics are cats and dogs. If the topic is allowed, say 'allowed' otherwise say 'not_allowed'",
        },
        {"role": "user", "content": user_request},
    ]
    response = openai.chat.completions.create(
        model=GPT_MODEL, messages=messages, temperature=0
    )

    print("Got guardrail response")
    return response.choices[0].message.content


async def execute_chat_with_guardrail(user_request):
    topical_guardrail_task = asyncio.create_task(topical_guardrail(user_request))
    chat_task = asyncio.create_task(get_chat_response(user_request))

    while True:
        done, _ = await asyncio.wait(
            [topical_guardrail_task, chat_task], return_when=asyncio.FIRST_COMPLETED
        )
        if topical_guardrail_task in done:
            guardrail_response = topical_guardrail_task.result()
            if guardrail_response == "not_allowed":
                chat_task.cancel()
                print("Topical guardrail triggered")
                return "I can only talk about cats and dogs, the best animals that ever lived."
            elif chat_task in done:
                chat_response = chat_task.result()
                return chat_response
        else:
            await asyncio.sleep(0.1)  # sleep for a bit before checking the tasks again

In [23]:
# Call the main function with the good request - this should go through
response = await execute_chat_with_guardrail(good_request)
print(response)

Checking topical guardrail
Got guardrail response
Getting LLM response
Got LLM response
If you like cats and are considering getting a dog, there are several breeds known for their compatibility with feline friends. Here are some of the best dog breeds that tend to get along well with cats:

1. **Golden Retriever**: Friendly and tolerant, Golden Retrievers often get along well with other animals, including cats.

2. **Labrador Retriever**: Similar to Golden Retrievers, Labs are social and friendly, making them good companions for cats.

3. **Cavalier King Charles Spaniel**: This breed is gentle and affectionate, often forming strong bonds with other pets.

4. **Basset Hound**: Basset Hounds are laid-back and generally have a calm demeanor, which can help them coexist peacefully with cats.

5. **Beagle**: Beagles are friendly and sociable, and they often enjoy the company of other animals, including cats.

6. **Pug**: Pugs are known for their playful and friendly nature, which can make 

In [24]:
# Call the main function with the bad request - this should get blocked
response = await execute_chat_with_guardrail(bad_request)
print(response)

Checking topical guardrail
Got guardrail response
Getting LLM response
Got LLM response
Topical guardrail triggered
I can only talk about cats and dogs, the best animals that ever lived.


ガードレールが機能しているようです - 最初の質問は通されましたが、2番目の質問はトピックから外れているためブロックされました。今度はこの概念を拡張して、LLMから得られるレスポンスも同様にモデレートしてみましょう。

## 2. 出力ガードレール

出力ガードレールは、LLMが返す内容を制御します。これらは多くの形態を取ることができ、最も一般的なものには以下があります：

- **幻覚/事実確認ガードレール：** 正確な情報のコーパスや幻覚的な応答の訓練セットを使用して、幻覚的な応答をブロックします。
- **モデレーションガードレール：** ブランドや企業のガイドラインを適用してLLMの結果をモデレートし、ガイドラインに違反する場合は応答をブロックまたは書き換えます。
- **構文チェック：** LLMからの構造化された出力は破損したり、パースできない状態で返される可能性があります - これらのガードレールはそれを検出し、再試行するか適切に失敗し、下流のアプリケーションでの障害を防ぎます。
    - これは関数呼び出しで適用される一般的な制御で、LLMが`function_call`を返す際に`arguments`で期待されるスキーマが返されることを保証します。

### モデレーションガードレール

ここでは、[G-Eval](https://arxiv.org/abs/2303.16634)評価手法のバージョンを使用して、LLMの応答における不要なコンテンツの存在をスコア化する**モデレーションガードレール**を実装します。この手法は、他の[ノートブック](https://github.com/openai/openai-cookbook/blob/main/examples/evaluation/How_to_eval_abstractive_summarization.ipynb)でより詳細に実演されています。

これを実現するために、`domain`を受け取り、一連の`steps`を使用して`content`の一部に`criteria`を適用するコンテンツモデレーションのための拡張可能なフレームワークを作成します：

1. ドメイン名を設定し、モデレートするコンテンツのタイプを記述します。
2. コンテンツに含まれるべきものと含まれるべきでないものを明確に概説する基準を提供します。
3. LLMがコンテンツを評価するためのステップバイステップの指示を提供します。
4. LLMは1-5の離散的なスコアを返します。

### ガードレール閾値の設定

私たちの出力ガードレールはLLMの応答を評価し、3以上のスコアのものをブロックします。この閾値の設定は最適化の一般的な領域です - 評価セットを構築し、混同行列を使用して結果を評価し、ガードレールに適切な許容度を設定することをお勧めします。ここでのトレードオフは一般的に以下の通りです：

- 偽陽性が多いと、顧客が苛立ち、アシスタントがあまり役に立たないように見える、断片的なユーザー体験につながります。
- 偽陰性が多いと、人々がアシスタントに不適切な質問に答えさせたり、プロンプトインジェクション/ジェイルブレイクを行ったりして、ビジネスに永続的な害をもたらす可能性があります。

例えば、ジェイルブレイクに対しては非常に低い閾値を設定したい場合があります。なぜなら、LLMがハイジャックされて危険なコンテンツを生成し、それがソーシャルメディアに掲載された場合のビジネスへのリスクが非常に高いからです。しかし、私たちのユースケースでは、いくつかの偽陰性を受け入れる用意があります。最悪の場合でも、誰かがラブラドールの方が適していたかもしれないビション・フリーゼを飼うことになるだけで、悲しいことではありますが、おそらく私たちのビジネスに永続的な損害を与えることはないでしょう（そう願っています）。

In [16]:
domain = "animal breed recommendation"

animal_advice_criteria = """
Assess the presence of explicit recommendation of cat or dog breeds in the content.
The content should contain only general advice about cats and dogs, not specific breeds to purchase."""

animal_advice_steps = """
1. Read the content and the criteria carefully.
2. Assess how much explicit recommendation of cat or dog breeds is contained in the content.
3. Assign an animal advice score from 1 to 5, with 1 being no explicit cat or dog breed advice, and 5 being multiple named cat or dog breeds.
"""

moderation_system_prompt = """
You are a moderation assistant. Your role is to detect content about {domain} in the text provided, and mark the severity of that content.

## {domain}

### Criteria

{scoring_criteria}

### Instructions

{scoring_steps}

### Content

{content}

### Evaluation (score only!)
"""

In [17]:
async def moderation_guardrail(chat_response):
    print("Checking moderation guardrail")
    mod_messages = [
        {"role": "user", "content": moderation_system_prompt.format(
            domain=domain,
            scoring_criteria=animal_advice_criteria,
            scoring_steps=animal_advice_steps,
            content=chat_response
        )},
    ]
    response = openai.chat.completions.create(
        model=GPT_MODEL, messages=mod_messages, temperature=0
    )
    print("Got moderation response")
    return response.choices[0].message.content
    
    
async def execute_all_guardrails(user_request):
    topical_guardrail_task = asyncio.create_task(topical_guardrail(user_request))
    chat_task = asyncio.create_task(get_chat_response(user_request))

    while True:
        done, _ = await asyncio.wait(
            [topical_guardrail_task, chat_task], return_when=asyncio.FIRST_COMPLETED
        )
        if topical_guardrail_task in done:
            guardrail_response = topical_guardrail_task.result()
            if guardrail_response == "not_allowed":
                chat_task.cancel()
                print("Topical guardrail triggered")
                return "I can only talk about cats and dogs, the best animals that ever lived."
            elif chat_task in done:
                chat_response = chat_task.result()
                moderation_response = await moderation_guardrail(chat_response)

                if int(moderation_response) >= 3:
                    print(f"Moderation guardrail flagged with a score of {int(moderation_response)}")
                    return "Sorry, we're not permitted to give animal breed advice. I can help you with any general queries you might have."

                else:
                    print('Passed moderation')
                    return chat_response
        else:
            await asyncio.sleep(0.1)  # sleep for a bit before checking the tasks again

In [18]:
# Adding a request that should pass both our topical guardrail and our moderation guardrail
great_request = 'What is some advice you can give to a new dog owner?'

In [19]:
tests = [good_request,bad_request,great_request]

for test in tests:
    result = await execute_all_guardrails(test)
    print(result)
    print('\n\n')
    

Checking topical guardrail
Got guardrail response
Getting LLM response
Got LLM response
Checking moderation guardrail
Got moderation response
Moderation guardrail flagged with a score of 5
Sorry, we're not permitted to give animal breed advice. I can help you with any general queries you might have.



Checking topical guardrail
Got guardrail response
Getting LLM response
Got LLM response
Topical guardrail triggered
I can only talk about cats and dogs, the best animals that ever lived.



Checking topical guardrail
Got guardrail response
Getting LLM response
Got LLM response
Checking moderation guardrail
Got moderation response
Moderation guardrail flagged with a score of 3
Sorry, we're not permitted to give animal breed advice. I can help you with any general queries you might have.





## まとめ

ガードレールはLLMにおいて活発で進化し続けるトピックであり、このノートブックがガードレールの中核概念について効果的な導入となったことを願っています。要約すると：

- ガードレールは、有害なコンテンツがアプリケーションやユーザーに到達することを防ぎ、本番環境でのLLMに操縦性を追加することを目的とした検出制御です。
- ガードレールは、LLMに到達する前のコンテンツを対象とする入力ガードレールと、LLMの応答を制御する出力ガードレールの形を取ることができます。
- ガードレールの設計と閾値の設定は、精度、レイテンシ、コストの間のトレードオフです。あなたの決定は、ガードレールのパフォーマンスの明確な評価と、偽陰性と偽陽性があなたのビジネスにとってどのようなコストになるかの理解に基づくべきです。
- 非同期設計原則を採用することで、ガードレールの数と範囲が増加してもユーザーへの影響を最小限に抑えるために、ガードレールを水平方向にスケールできます。

皆さんがこれをどのように発展させ、エコシステムが成熟するにつれてガードレールに関する考え方がどのように進化するかを楽しみにしています。